### Optical flow algorithm explained in: https://www.ipol.im/pub/art/2013/26/article.pdf

In [2]:
import numpy as np
import os
from matplotlib import pyplot as plt
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D
import cv2
import skimage.io as io
from copy import deepcopy, copy
import time

In [3]:
#downsize all images by factor of 100 (already done):
# img_files_path = [_ for _ in os.listdir(r'\\fatherserverdw\Kevin\imageregistration\registered_images') if _.endswith(".png")]
# img_files_path_complete = [os.path.join(r'\\fatherserverdw\Kevin\imageregistration\registered_images', x) for x in img_files_path]

# ex_img = Image.open(img_files_path_complete[0])
# scale_percent = 1
# width = int(ex_img.size[1] * scale_percent / 100)
# height = int(ex_img.size[0] * scale_percent / 100)
# dim = (width, height)
# for idx in range(len(img_files_path_complete)):
#     img = Image.open(img_files_path_complete[idx])
#     img_ra = np.array(img)
#     resize_img = cv2.resize(img_ra,dim,interpolation = cv2.INTER_AREA)
#     resize_img = cv2.cvtColor(resize_img, cv2.COLOR_BGR2RGB)
#     dstpth = os.path.join(r'\\fatherserverdw\Kevin\imageregistration\registered_images\downsized',img_files_path[idx])
#     cv2.imwrite(dstpth,resize_img)

In [4]:
resize_img_files_path = [_ for _ in os.listdir(r'\\fatherserverdw\Kevin\imageregistration\registered_images\downsized') if _.endswith(".png")]
resize_img_files_path_complete = [os.path.join(r'\\fatherserverdw\Kevin\imageregistration\registered_images\downsized', x) for x in resize_img_files_path]

In [5]:
# sample_ig = cv2.imread(r'C:\Users\choke\Desktop\z0004_4c1.png')
# sample_ig1 = cv2.imread(r'C:\Users\choke\Desktop\z0007_7c1.png')
# sample_ig_path = r'C:\Users\choke\Desktop\z0004_4c1.png'
# sample_ig1_path = r'C:\Users\choke\Desktop\z0007_7c1.png'
# cv2.imwrite(r'C:\Users\choke\Desktop\z0004_4c1.tif',sample_ig)
# tif_pic = Image.open(r'C:\Users\choke\Desktop\z0004_4c1.tif')
im_list = io.imread_collection(resize_img_files_path_complete)
im_3d = im_list.concatenate()

# Try 3DSlicer in Python

In [7]:
from vedo import *
vol = Volume(im_3d)
vol.print()
show(vol, axes=2)

TypeError: SetDimensions argument 1: expected a sequence of 3 values, got 4 values

In [7]:
from vedo import Picture, dataurl
pic = Picture(dataurl+'dog.jpg').pad()
pic.append([pic,pic,pic], axis='y')
pic.append([pic,pic,pic,pic], axis='x')
pic.show(axes=1).close()

In [1]:
#first build mesh from image
#then use mesh

In [5]:
from vedo import build_lut, Sphere, show, settings

settings.use_depth_peeling = True # might help with transparencies

# generate a sphere and stretch it, so it sits between z=-2 and z=+2
mesh = Sphere(quads=True).scale([1,1,2]).linewidth(0.1)

# create some dummy data array to be associated to points
data = mesh.points()[:,2]  # pick z-coords, use them as scalar data
data[10:70] = float('nan') # make some values invalid by setting to NaN
data[300:600] = 100        # send some values very far above-scale

# build a custom Look-Up-Table of colors:
#     value, color, alpha
lut = build_lut(
    [
      #(-2, 'pink'      ),  # up to -2 is pink
      (0.0, 'pink'      ),  # up to 0 is pink
      (0.4, 'green', 0.5),  # up to 0.4 is green with alpha=0.5
      (0.7, 'darkblue'  ),
      #( 2, 'darkblue'  ),
    ],
    vmin=-1.2,
    vmax= 0.7,
    below_color='lightblue',
    above_color='grey',
    nan_color='red',
    interpolate=False,
)
# 3D scalarbar:
mesh.cmap(lut, data).add_scalarbar3d(title='My 3D scalarbar', c='white')
mesh.scalarbar.scale(1.5).rotate_x(90).y(1) # make it bigger and place it

# 2D scalarbar:
# mesh.cmap(lut, data).add_scalarbar()

show(mesh, __doc__,
     axes=dict(zlabel_size=.04, number_of_divisions=10),
     elevation=-80, bg='blackboard',
).close()

In [6]:
from vedo import Picture, dataurl, show
import numpy as np
pic = Picture(resize_img_files_path_complete[0])
msh = pic.tomesh()  # make a quad-mesh out of it

# build a scalar array with intensities
rgb = msh.pointdata["RGBA"]
intensity = np.sum(rgb, axis=1)
intensityz = np.zeros_like(rgb)
intensityz[:,2] = intensity / 10

# set the new vertex points
pts = msh.points() + intensityz
msh.points(pts)

# more cosmetics
msh.triangulate().smooth()
msh.lighting("default").linewidth(0)
msh.cmap("bone", "RGBA").add_scalarbar()

msht = pic.clone().threshold(100).linewidth(0)

show([[pic, "A normal jpg image.."],
      [msh, "..becomes a polygonal Mesh"],
      [msht, "Thresholding also generates a Mesh"]
     ], N=3, axes=1, zoom=1.1, elevation=-20, bg='black')

In [ ]:
im_list = io.imread_collection(resize_img_files_path_complete)
im_3d = im_list.concatenate()
im_3d.shape
# im_3d_new = im_3d.transpose(1,2,0,3)

imlist = im_3d
binaries = []
colors = []

start = time.time()

for im in imlist:
    binary = im[:,:,1]>5 #230
    binaries.append(binary)
    color = deepcopy(im)
    color[~binary] = [0,0,0]
    color = color/255
    color = color.astype(np.object)
    colors.append(color)
binaries = np.array(binaries)
colors = np.array(colors)

ax = plt.figure().add_subplot(projection='3d')
ax.voxels(binaries, facecolors=colors, edgecolors = colors, shade=None)
end = time.time()
print("time it took to register: "+  str(end-start) + " seconds")

plt.show()

In [ ]:
# plot one image first:
test_im_list = resize_img_files_path_complete
test_im_list = io.imread_collection(test_im_list)
test_im_3d = test_im_list.concatenate()

#compile tissue binary masks and their rgb values
imlist = test_im_3d
binaries = []
colors = []
for im in imlist[0:4]:
    binary = (150 < im[:, :, 0]) & (im[:, :, 1] < 210)
    binaries.append(binary)
    color = copy(im)
    color[~binary] = [0,0,0]
    color = color/255
    # a_ra = np.ones(color.shape) * 0.5
    # color = np.dstack((color,a_ra[:,:,1])) #rgba
    colors.append(color)

In [ ]:
binaries = np.array(binaries)
colors = np.array(colors)
binaries.shape,colors.shape #N,X,Y,C

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
voxel_obj = ax.voxels(binaries, facecolors=colors, edgecolors = 'k', shade=False)
# plt.show()

In [ ]:
from skimage.morphology import closing, square, remove_small_objects, remove_small_holes
from skimage.segmentation import clear_border

In [ ]:
bw = (150 < im[:, :, 0]) & (im[:, :, 1] < 210)
bw = closing(bw, square(3))
## below lines could be useful to post process tissue binary mask
# minTA = 900 #resolution sensitive
# bw = remove_small_objects(bw, min_size=minTA, connectivity=2)
# minTAhole = 40 #resolution sensitive
# bw = remove_small_holes(bw, area_threshold=minTAhole)
# bw = clear_border(bw)
Image.fromarray(bw)

In [ ]:
# start = time.time()
#
#
# imlist = []
# binaries = []
# colors = []
# for im in imlist:
#     binary = im[:,:,1]>230
#     binaries.append(binary)
#     color = deepcopy(im)
#     color[~binary] = [0,0,0]
#     color = color.astype(np.object)
#     colors.append(color)
# binaries = np.array(binaries)
# colors = np.array(colors)
#
# ax = plt.figure().add_subplot(projection='3d')
# ax.voxels(binaries, facecolors=colors, edgecolor='k')
#
#
# end = time.time()
# print("time it took to register: "+  str(end-start) + " seconds")

In [ ]:

# start = time.time()
#
# X, Y= np.mgrid[-3164:3164:1, -2676:2676:1]
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# for i in np.linspace(0,33, num=34):
#     Z = np.zeros(X.shape)+i
#     idx = int(i)
#     ig = Image.open(img_files_path_complete[idx])
#     ig = np.array(ig).astype(np.float32)
#     ig = ig / 255
#     T = ig
#     final_fig = ax.plot_surface(X, Y, Z, facecolors=T, linewidth=0, alpha = 0.5, cstride
#         = 10, rstride = 10)
# plt.show()
#
# end = time.time()
# print("time it took to register: "+  str(end-start) + " seconds")
# #save finalfig

In [ ]:
# from mayavi import mlab
# from mayavi.mlab import surf
# start = time.time()
#
# X, Y = np.mgrid[-3164:3164:1, -2676:2676:1]
#
# for i in np.linspace(0,33, num=34):
#     Z = np.zeros(X.shape)+i
#     idx = int(i)
#     ig = Image.open(img_files_path_complete[idx])
#     ig = np.array(ig).astype(np.float32)
#     ig = ig / 255
#     ig = tuple(map(tuple, ig))
#     clr = ig
# surf(X,Y,Z,color = clr,warp_scale = "auto")
# mlab.show()
#
# end = time.time()
# print("time it took to register: "+  str(end-start) + " seconds")